# Add Source Data to intermediate_data_01

## I. Essentials

#### Check directories and load library packages

Working directory is `project_folder/data/raw_data/source_data`.

In [ ]:
current_dir = getwd()
source_data_dir = "../../data/raw_data/source_data"
setwd(source_data_dir)

Load essential library packages

In [62]:
library("readxl")
library("dplyr")

In [47]:
getwd()

[1] "/scratch/research/projects/chlamydomonas/lipid_selection/data/raw_data/source_data"

#### Create `intermediate_data_01` dataframe

In [192]:
intermediate_data_01 <- data.frame(
    gene_id = character(),
    gene_name = character(),
    protein_id = character(),
    protein_name = character(),
    first_author = character(),
    publication_year = numeric(),
    genome_version = numeric(),
    database_source = character(),
    inclusion_criteria = character(),
    candidate_gene = logical(),
    stringsAsFactors = FALSE)
int_data_colnames <- colnames(intermediate_data_01)
intermediate_data_01

gene_id,gene_name,protein_id,protein_name,first_author,publication_year,genome_version,database_source,inclusion_criteria,candidate_gene
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<lgl>


#### refresh_int_df_01

In [ ]:
#Recreates an empty `intermediate_data_01` dataframe
#Useful when working dataset by dataset without overloading memory



#### check_excel_data

In [211]:
#Check if excel dataset has more than one sheet. 
#Check if file type is .xls or .xlsx
check_excel_data <- function (file_path){
    
    #If file type is not .xls or .xlsx, return FALSE
    #Return sheet name(s) as a list of strings
    
    #check if "readxl" is loaded
    require("readxl") 
    
    #check if file type is .xls or .xlsx. excel_sheets() only works with these file types
    if (strsplit(file_path, "[.]")[[1]][2] %in% c("xls", "xlsx")){
        list_of_sheets <- excel_sheets(file_path)
        return (list_of_sheets)
    }
    else {
        stop("File type is not .xls or .xlsx")} 
    
}

In [212]:
#Test code with excel dataset with more than one sheet
check_excel_data("Bajhaiya_2016.xls")  

#Test code with excel dataset with one sheet
check_excel_data("Boyle_2012.xls") 

#Test code with incorrect file type
check_excel_data("Li_2016.xlsb")

[1] "Day 3" "Day 5"

[1] "Sheet1"

ERROR: Error in check_excel_data("Li_2016.xlsb"): File type is not .xls or .xlsx


#### import_messy_excel

In [217]:
#Remove non-data table rows and import a cleaner dataframe from an excel dataset

import_messy_excel <- function(file_path, sheet_name){
    
    #check if "readxl" is loaded
    require("readxl") 
    require("dplyr")
    require("stringr")
    
    #Checks to see if sheet name exists
    if (sheet_name %in% excel_sheets(file_path) == FALSE){
            stop("Sheet name does not exist")
    }
    
    #Remove rows in excel sheet if more than half of the columns have NAs
    df <- read_excel(file_path, sheet = sheet_name) %>% 
        filter(rowSums(is.na(.))/ncol(.) < 0.5)
    
    #Assumes Row 1 of the subset dataframe is the column names
    #Assign Row 1 as column names and remove Row 1
    colnames(df)<- df[1,] %>% str_replace_all(" ", "_")
    df<- df[-1,]
    message("Check if column names are correct.")
    
    return(df)
    
}

In [218]:
#Test code with the correct excel dataset
df <- import_messy_excel("Bajhaiya_2016.xls", "Day 3") 
print(df[1:3,])

#Test code with incorrect excel sheet name
df <- import_messy_excel("Bajhaiya_2016.xls", "Wrong name") 

New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* … and 13 more problems
Check if column names are correct.


# A tibble: 3 x 19
  Gene_id Gene_name Description WT_HP_D3 WT_LP_D3 psr1_HP_D3 psr1_LP_D3
  <chr>   <chr>     <chr>       <chr>    <chr>    <chr>      <chr>     
1 Cre09.… Cre09.g4… ---         76.6888… 7263.06… 1.18872    4.2062200…
2 Cre04.… PHOX      alkaline p… 167.419… 14234.5  3.5661499… 4.2062200…
3 Cre01.… Cre01.g0… ---         6.48073… 1093.39… 1.18872    0         
# … with 12 more variables: WT_HP_D3_DESeq_norm <chr>,
#   WT_LP_D3_DESeq_norm <chr>, psr1_HP_D3_DESeq_norm <chr>,
#   psr1_LP_D3_DESeq_norm <chr>, `WT_HP_D3_DESeq__log2+10` <chr>,
#   `WT_LP_D3_DESeq__log2+10` <chr>, `psr1_HP_D3_DESeq__log2+10` <chr>,
#   `psr1_LP_D3_DESeq__log2+10` <chr>, `WT_LP-HP_D3_logdiff` <chr>,
#   `psr1_LP-HP_D3_logdiff` <chr>, `WT_LP_/_HP_D3_foldchange` <chr>,
#   `psr1_LP_/_HP_D3_foldchange` <chr>


ERROR: Error in import_messy_excel("Bajhaiya_2016.xls", "Wrong name"): Sheet name does not exist


#### append_genes

In [199]:
#Add genes to intermediate_data_01
#Input: two dataframes - candidate_genes, not_candidate_genes

append_genes <- function(gene_data = gene_data, basic_info = basic_info, df = intermediate_data_01){
    
    require("dplyr")
    
    clean_data <- merge(gene_data, basic_info) %>% 
        mutate(protein_id = NA,
              protein_name = NA)
    
    intermediate_data_01 <- rbind(df, clean_data)
    
    return(intermediate_data_01)
    
}

## II. Add source data to intermediate_data_01

### Bajhaiya_2016.xls


In [160]:
#Basic information

file_name = "Bajhaiya_2016.xls"
genome_version = 5.3
database_source = "Phytozome 9.1"
inclusion_criteria = "fold difference greater than 2"
first_author = "Bajhaiya"
publication_year = 2016

basic_info <- data.frame(genome_version, database_source, inclusion_criteria, first_author, publication_year, stringsAsFactors = FALSE)
str(basic_info)

'data.frame':	1 obs. of  5 variables:
 $ genome_version    : num 5.3
 $ database_source   : chr "Phytozome 9.1"
 $ inclusion_criteria: chr "fold difference greater than 2"
 $ first_author      : chr "Bajhaiya"
 $ publication_year  : num 2016


In [161]:
#Check file type and number of excel sheets

sheets <- check_excel_data(file_name)  
print(length(sheets))

[1] 2


#### 1. Collect candidate genes based on inclusion criteria.

**Inclusion criteria**: Within each strain, if fold-difference between high P and low P is >2 count as candidate gene. 

**Sheet 1: "Day 3" **

In [163]:
df <- import_messy_excel(file_name, sheets[1]) 
#Calculate fold difference using normalized expression
temp1 <- df %>% 
    select(starts_with("Gene"), ends_with("norm")) %>%
    mutate_at(vars(ends_with("norm")),list(as.numeric)) %>%
    filter(WT_HP_D3_DESeq_norm/WT_LP_D3_DESeq_norm >= 2 |
          WT_LP_D3_DESeq_norm/WT_HP_D3_DESeq_norm >= 2 |
          psr1_HP_D3_DESeq_norm/psr1_LP_D3_DESeq_norm >= 2 |
          psr1_LP_D3_DESeq_norm/psr1_HP_D3_DESeq_norm >= 2) %>%
    select(starts_with("Gene"))
           
temp2[1:5,]

New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* … and 13 more problems


Gene_id,Gene_name
<chr>,<chr>
Cre09.g404900,Cre09.g404900
Cre04.g216700,PHOX
Cre01.g044300,Cre01.g044300
g16424,g16424
g2975,MPA1


** Sheet 2: "Day 5" **

In [164]:
df <- import_messy_excel(file_name, sheets[2]) 
#Calculate fold difference using normalized expression
temp2 <- df %>% 
    select(starts_with("Gene"), ends_with("norm")) %>%
    mutate_at(vars(ends_with("norm")),list(as.numeric)) %>%
    filter(WT_HP_D5_DESeq_norm/WT_LP_D5_DESeq_norm >= 2 |
          WT_LP_D5_DESeq_norm/WT_HP_D5_DESeq_norm >= 2 |
          psr1_HP_D5_DESeq_norm/psr1_LP_D5_DESeq_norm >= 2 |
          psr1_LP_D5_DESeq_norm/psr1_HP_D5_DESeq_norm >= 2) %>%
    select(starts_with("Gene"))
temp2[1:5,]

New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* … and 13 more problems


Gene_id,Gene_name
<chr>,<chr>
Cre09.g404900,Cre09.g404900
Cre04.g216700,PHOX
Cre01.g044300,Cre01.g044300
g16424,g16424
g2975,MPA1


#### 2. Join Sheet 1 and 2 data by creating two dataframes:
* `candidate_genes`
* `not_candidate_genes`

In [188]:
candidate_temp <- left_join(temp1, temp2, by = c("Gene_id", "Gene_name")) 

not_candidate_genes_df <- anti_join(df, candidate_temp) %>% select(starts_with("Gene")) %>%
    mutate(candidate_gene = FALSE)

candidate_genes <- candidate_temp %>% 
    mutate(candidate_gene = TRUE)

gene_data <- rbind(candidate_genes, not_candidate_genes_df) %>%
    rename(gene_id = Gene_id,
          gene_name = Gene_name)

gene_data[1:5,]

Joining, by = c("Gene_id", "Gene_name")


gene_id,gene_name,candidate_gene
<chr>,<chr>,<lgl>
Cre09.g404900,Cre09.g404900,TRUE
Cre04.g216700,PHOX,TRUE
Cre01.g044300,Cre01.g044300,TRUE
g16424,g16424,TRUE
g2975,MPA1,TRUE


#### 3. Append to intermediate_data_01

In [201]:
intermediate_data_01<- append_genes(gene_data = gene_data, basic_info = basic_info, df = intermediate_data_01)
intermediate_data_01[1:5,]

gene_id,gene_name,candidate_gene,genome_version,database_source,inclusion_criteria,first_author,publication_year,protein_id,protein_name
<chr>,<chr>,<lgl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
Cre09.g404900,Cre09.g404900,TRUE,5.3,Phytozome 9.1,fold difference greater than 2,Bajhaiya,2016,NA,NA
Cre04.g216700,PHOX,TRUE,5.3,Phytozome 9.1,fold difference greater than 2,Bajhaiya,2016,NA,NA
Cre01.g044300,Cre01.g044300,TRUE,5.3,Phytozome 9.1,fold difference greater than 2,Bajhaiya,2016,NA,NA
g16424,g16424,TRUE,5.3,Phytozome 9.1,fold difference greater than 2,Bajhaiya,2016,NA,NA
g2975,MPA1,TRUE,5.3,Phytozome 9.1,fold difference greater than 2,Bajhaiya,2016,NA,NA


In [200]:
intermediate_data_01 <- data.frame(
    gene_id = character(),
    gene_name = character(),
    protein_id = character(),
    protein_name = character(),
    first_author = character(),
    publication_year = numeric(),
    genome_version = numeric(),
    database_source = character(),
    inclusion_criteria = character(),
    candidate_gene = logical(),
    stringsAsFactors = FALSE)
int_data_colnames <- colnames(intermediate_data_01)
intermediate_data_01

gene_id,gene_name,protein_id,protein_name,first_author,publication_year,genome_version,database_source,inclusion_criteria,candidate_gene
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<lgl>


## III. Add cleaned source data to intermediate_data_01